In [78]:
import weaviate
import weaviate.classes.config as wvcc
from weaviate.classes.config import Property, DataType, Configure
import requests
import json

# Cloud Connection

In [81]:
#define a client 
client = weaviate.connect_to_wcs(
    cluster_url="https://ep9buqisqh2agxcqhaqja.c0.us-central1.gcp.weaviate.cloud",
    auth_credentials=weaviate.classes.init.Auth.api_key("cBbDQMWkOgPEzzQYg2megPVbhrZ28P6mH33k"),
)

#checking client status
def check_client(client):
    try:
        # check client is ready or not
        response = client.is_ready()
        if response:
            print("Weaviate client is ready.")
        else:
            print("Weaviate client is not ready.")
    except Exception as e:
        print(f"Error checking Weaviate client status: {e}")
        
    
check_client(client)

Weaviate client is ready.


# Collection making

In [25]:
# create a basic collection without victorizer

client.collections.create(
    "new_test_collection",
    vectorizer_config= Configure.Vectorizer.text2vec_huggingface(model = "sentence-transformers/all-MiniLM-L6-v2")
)

collection = client.collections.get('new_test_collection')
collection

# Insert data 

In [29]:
def insert_data(client, url = 'https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json'):
    # fetch data from the repo
    repo = requests.get(url)
    data = json.loads(repo.text)
    
    # prepare our data
    question_objs = []
    for i, d in enumerate(data):
        question_objs.append({
            "answer" : d['Answer'],
            "question" : d['Question'],
            "category" : d['Category']
        })
        
    # insert it and store
    questions = client.collections.get("new_test_collection")
    questions.data.insert_many(question_objs)
    
    
insert_data(client)

# Query to show the data 

In [82]:

try:
    pass # Replace with your code. Close client gracefully in the finally block.
    questions = client.collections.get("Question")

    response = questions.query.near_text(
        query="biology",
        limit=2
    )

    print(response.objects[0].properties)  # Inspect the first object

finally:
    client.close()  # Close client gracefully

WeaviateQueryError: Query call with protocol GRPC search failed with message <AioRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "could not find class Question in schema"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"could not find class Question in schema", grpc_status:2, created_time:"2024-08-17T03:30:43.9287791+00:00"}"
>.